## Oppstart

In [1]:

import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)


# Eksempel

I eksemplet ser vi først på en tilfeldig bok, deretter mulighetene for å gjøre det samme med et helt korpus.

## Korpus

Bygg først et korpus, enten fra kommando `nb.book_corpus` eller `book_urn`, eller gjenskap fra fil.

In [7]:
korpus = nb.book_corpus(author='hamsun%')

In [ ]:
#korpus = nb.restore_metadata_from_excel('metadatafil.xls')

## Sjekk innholdet

Gjør noen søk for å se at alt er ok. Konkordanser er fine til det. Her går urnene gjennom `nb.pure_urn` -  i fremtiden skal det ikke være nødvendig.

In [4]:
nb.urn_concordance(word = 'arbeid', urns = korpus)

"Hamsuns beste, Hamsun, Knut, 2006",kunne han skrive om sitt,arbeid,: « Dog maa jeg
"Hamsuns beste, Hamsun, Knut, 2006",han tusen kroner til Nansens,arbeid,for sultende barn
"Hamsuns beste, Hamsun, Knut, 2006",Jeg anførte følgende : Mitt,arbeid,i Gyldendal var meget vanskelig
"Hamsuns beste, Hamsun, Knut, 2006",", og han tar seg",arbeid,på prærien .
"Samlede verker, Hamsun, Knut, 2009",av dette innfall i mitt,arbeid,", går jeg i gang"
"Samlede verker, Hamsun, Knut, 2009",bredere plass i det politiske,arbeid,. Ut fra forskjellige livssyn
"Samlede verker, Hamsun, Knut, 2009",i 1934 ble et slikt,arbeid,igangsatt - et arbeid som
"Samlede verker, Hamsun, Knut, 2009",slikt arbeid igangsatt - et,arbeid,som førte frem til den
"Bygdebarn, Hamsun, Marie, 1942",Det hadde vært et svært,arbeid,"å bygge hyttene , for"


## Navn

Finn forslag til navn med `names`. Se på dem med `show_names`. Her er det muligheter til å skru litt på parametrene. Hvis ratioen skrus ned fjernes en del ord, og om cutoff settes høyere vil også enkelte navneforslag kunne forsvinne.

Velg hvilken bok som skal studeres (alle kan tas, men se under). Se på listen over korpus og sett den indeksen som passer. Her er det bare å føle seg frem.

In [ ]:
korpus

Finn korpuset ved å sette inn en index tilsvarende den som står helt til venstre i listen over.

In [ ]:
korpus.loc[2]

Og knytt det til variabelen BOOK, eventuelt kopier ut URN og ta det derfra.

In [ ]:
# sett inn med indeks

BOOK = nb.pure_urn(korpus['urn'])[1]

# sett inn med URN fra listen

#BOOK = 2007021401006 

Hent ut navn fra boken

In [ ]:
navn = nb.names(BOOK, ratio=0.5, cutoff=10)
tm.show_names(navn)

### Lagring og redigering

Rediger filen fra cellen med lagringskommando (`names_to_token_map_file(....)`)
Listen trenger litt redigering, og sammenslåing. Lagre til Excel, og benytt `orient = column` for å ha navn i kolonnner, eller `orient = row` for å redigere radvis. Noen store filer med over 256 navn krever `row`. Kommandoen sjekker om filen allerede fins, og vil ikke skrive over.

In [ ]:
tm.names_to_token_map_file(navn, filename = "en_bok15.xls", orient = "column")

### Redigeringsaktivitet

Åpne i for eksempel i excel eller tilsvarende som for eksempel openoffice eller libreoffice. Etter redigering, kjøres cellen under. Husk bare å benytte riktig filnavn. 

Husk at verdien på orient må være lik mellom lagring og lesing av excelfil.

In [ ]:
redigerte_navn = tm.read_token_map_file("en_bok15.xls", orient = "column")

Listen med redigerte navn går inn i grafen - vi tar med oss variabelen BOOK (se øverst):

In [ ]:
G = tm.character_network(BOOK, redigerte_navn)

Dernest kan den vises og analyseres med passende kommandoer

In [ ]:
gnl.show_graph(G)

In [ ]:
gnl.show_communities(G)